In [ ]:
import sys
import os
import pandas as pd
import urllib

# Set the Python path to the location of the directory containing "immport_download.py"
immport_download_code = '/home/huz/deep_learning_CyTOF/bin'
sys.path.insert(0,immport_download_code)
os.chdir(immport_download_code)

import immport_download

In [ ]:
cytof_files = pd.read_csv("../result_01_cytof_file_info.csv")
cytof_files.head()

In [ ]:
user_name = "your_immport_user_name"
password = "your_immport_password"
output_directory = "../CyTOF_data"
token = immport_download.request_immport_token(user_name, password)

file_info = pd.DataFrame()
for sdy in cytof_files.study_accession.unique():
    r = immport_download.api("https://api.immport.org/data/query/result"+
                         "/filePath?studyAccession="+sdy+
                         "&measurementTechnique=CyTOF",
                         token)
    df = pd.read_json(r)
    df = df[df['fileDetail'] == "CyTOF result"]
    df_fcs = df[['studyAccession','filePath']]
    file_info = file_info.append(df_fcs,ignore_index=True )
    print(file_info.shape)

unique_file_paths = file_info.filePath.unique()
unique_fcs_paths = [path for path in unique_file_paths 
                    if path.endswith((".fcs",".FCS"))]
len(unique_fcs_paths)


In [ ]:
##### skip fcs files that have already exist
unique_fcs_base = [os.path.basename(t1) for t1 in unique_fcs_paths]
existing_fcs = os.listdir(output_directory)
unique_fcs_paths =  [unique_fcs_paths[i] for i in range(0,len(unique_fcs_paths)) if unique_fcs_base[i] not in existing_fcs]

In [ ]:
for i in range(0,len(unique_fcs_paths)):
    print("Downloading: ",unique_fcs_paths[i])
    immport_download.download_file(user_name,password,
                                    unique_fcs_paths[i],output_directory)